# Imports & Prepare Data

In [1]:
from models.ml_sequential import prepare_data, run_all
import tensorflow as tf
# Disable GPU training, comment out to enable
tf.config.set_visible_devices([], 'GPU')
# Imports used across models
from keras.layers import Input, Dense, TimeDistributed, Dropout
from keras.models import Sequential
from keras.regularizers import l2

features = ['female', 'age', 'height', 'mass', 'ta_set', 'rh_set']
features_scaler, output_scaler, X_padded_folds, y_padded_folds, max_len = prepare_data(features)

# Define the input/output shape
input_shape = (None, X_padded_folds[0].shape[-1])
output_shape = y_padded_folds[0].shape[-1]

# Train the model with early stopping
from keras.callbacks import EarlyStopping

# Custom log callback
class PrintCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f'Epoch: {epoch+1}, loss: {logs["loss"]:.2f}', end='\r')

    def on_train_end(self, epoch):
        print()

Max sequence length: 660
Max sequence length: 660
Max sequence length: 660
Max sequence length: 660


# RNN

In [ ]:
from keras.layers import SimpleRNN

# Results from 16 sized model
# Average TRE RMSE: 0.40913350652486097
# Average MTSK RMSE: 1.2484756718098815

# MODEL NAME
model_name = 'ml_rnn'
model_sizes = [ 32, 64 ]

for model_size in model_sizes:
	models = []
	print("Model size:", model_size)
	for idx, (X_padded, y_padded) in enumerate(zip(X_padded_folds, y_padded_folds)):
		fold_number = idx + 1
		print("Fold:", fold_number)
		# Model architecture
		model = Sequential()
		model.add(Input(shape=input_shape))
		model.add(SimpleRNN(model_size, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01)))
		model.add(TimeDistributed(Dense(8, activation='linear', kernel_regularizer=l2(0.01))))
		model.add(Dropout(0.2))
		model.add(TimeDistributed(Dense(output_shape)))

		# Compile the model
		model.compile(loss='mse', optimizer='adam')

		# Train the model
		early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
		model.fit(X_padded, y_padded, validation_split=0.2, epochs=500, batch_size=32, callbacks=[PrintCallback(), early_stopping], verbose=0)

		# Save the model
		model.save('model_weights/{}-fold{}-size{}.keras'.format(model_name, fold_number, model_size))

		models.append(model)

	# Run simulations using trained model
	run_all(models, model_name, features, features_scaler, output_scaler, max_len)

Model size: 32
Fold: 1
Epoch: 346, loss: 0.03
Fold: 2
Epoch: 500, loss: 0.06
Fold: 3
Epoch: 449, loss: 0.04
Fold: 4
Epoch: 463, loss: 0.03
ml_rnn
Average TRE RMSE: 0.410779462041641
Average MTSK RMSE: 1.2443753602807686
Model size: 64
Fold: 1
Epoch: 383, loss: 0.03
Fold: 2
Epoch: 336, loss: 0.03
Fold: 3
Epoch: 474, loss: 0.03
Fold: 4
Epoch: 335, loss: 0.04
ml_rnn
Average TRE RMSE: 0.41461371511959283
Average MTSK RMSE: 1.2356653420277874


# LSTM

In [2]:
from keras.layers import LSTM

# Results from 16 sized model
# Average TRE RMSE: 0.4006921834540827
# Average MTSK RMSE: 1.1933293403769494

# MODEL NAME
model_name = 'ml_lstm'
model_sizes = [ 8, 32, 64 ]

for model_size in model_sizes:
	models = []
	print("Model size:", model_size)
	for idx, (X_padded, y_padded) in enumerate(zip(X_padded_folds, y_padded_folds)):
		fold_number = idx + 1
		print("Fold:", fold_number)
		# Model architecture
		model = Sequential()
		model.add(Input(shape=input_shape))
		model.add(LSTM(model_size, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01)))
		model.add(TimeDistributed(Dense(8, activation='linear', kernel_regularizer=l2(0.01))))
		model.add(Dropout(0.2))
		model.add(TimeDistributed(Dense(output_shape)))

		# Compile the model
		model.compile(loss='mse', optimizer='adam')

		# Train the model
		early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
		model.fit(X_padded, y_padded, validation_split=0.2, epochs=500, batch_size=32, callbacks=[PrintCallback(), early_stopping], verbose=0)

		# Save the model
		model.save('model_weights/{}-fold{}-size{}.keras'.format(model_name, fold_number, model_size))

		models.append(model)

	# Run simulations using trained model
	run_all(models, model_name, features, features_scaler, output_scaler, max_len)

Model size: 8
Fold: 1
Epoch: 285, loss: 0.04
Fold: 2
Epoch: 447, loss: 0.03
Fold: 3
Epoch: 353, loss: 0.04
Fold: 4
Epoch: 500, loss: 0.04
ml_lstm
Average TRE RMSE: 0.4002177547379054
Average MTSK RMSE: 1.2118397678835118
Model size: 32
Fold: 1
Epoch: 173, loss: 0.04
Fold: 2
Epoch: 305, loss: 0.03
Fold: 3
Epoch: 169, loss: 0.05
Fold: 4
Epoch: 267, loss: 0.04
ml_lstm
Average TRE RMSE: 0.40426242564629805
Average MTSK RMSE: 1.2130538083625308
Model size: 64
Fold: 1
Epoch: 192, loss: 0.04
Fold: 2
Epoch: 162, loss: 0.05
Fold: 3
Epoch: 152, loss: 0.04
Fold: 4
Epoch: 211, loss: 0.04
ml_lstm
Average TRE RMSE: 0.40716918527757723
Average MTSK RMSE: 1.219020673328306


# GRU

In [3]:
from keras.layers import GRU

# Results from 16 sized model
# Average TRE RMSE: 0.35102718032762875
# Average MTSK RMSE: 0.9882427786586373

# MODEL NAME
model_name = 'ml_gru'
model_sizes = [ 8, 32, 64 ]

for model_size in model_sizes:
	models = []
	print("Model size:", model_size)
	for idx, (X_padded, y_padded) in enumerate(zip(X_padded_folds, y_padded_folds)):
		fold_number = idx + 1
		print("Fold:", fold_number)
		# Model architecture
		model = Sequential()
		model.add(Input(shape=input_shape))
		model.add(GRU(model_size, return_sequences=True))
		model.add(TimeDistributed(Dense(8, activation='linear', kernel_regularizer=l2(0.01))))
		model.add(Dropout(0.2))
		model.add(TimeDistributed(Dense(output_shape)))

		# Compile the model
		model.compile(loss='mse', optimizer='adam')

		# Train the model
		early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
		model.fit(X_padded, y_padded, validation_split=0.2, epochs=500, batch_size=32, callbacks=[PrintCallback(), early_stopping], verbose=0)

		# Save the model
		model.save('model_weights/{}-fold{}-size{}.keras'.format(model_name, fold_number, model_size))

		models.append(model)

	# Run simulations using trained model
	run_all(models, model_name, features, features_scaler, output_scaler, max_len)

Model size: 8
Fold: 1
Epoch: 369, loss: 0.01
Fold: 2
Epoch: 500, loss: 0.02
Fold: 3
Epoch: 391, loss: 0.02
Fold: 4
Epoch: 500, loss: 0.02
ml_gru
Average TRE RMSE: 0.3702108755798637
Average MTSK RMSE: 1.047010564418845
Model size: 32
Fold: 1
Epoch: 324, loss: 0.01
Fold: 2
Epoch: 215, loss: 0.02
Fold: 3
Epoch: 345, loss: 0.01
Fold: 4
Epoch: 234, loss: 0.01
ml_gru
Average TRE RMSE: 0.34778583558998105
Average MTSK RMSE: 0.9631366050031559
Model size: 64
Fold: 1
Epoch: 226, loss: 0.02
Fold: 2
Epoch: 206, loss: 0.02
Fold: 3
Epoch: 273, loss: 0.02
Fold: 4
Epoch: 209, loss: 0.01
ml_gru
Average TRE RMSE: 0.33714177906760706
Average MTSK RMSE: 0.9706077939508453


# Select best performing model size

In [6]:
import keras

model_name = 'ml_gru'
model_size = 32

def get_results_for_model_size(model_name, model_size=None):
	models = []
	for i in range(4):
		fold_number = i + 1
		if model_size:
			model = keras.models.load_model(f'model_weights/{model_name}-fold{fold_number}-size{model_size}.keras')
		else:
			model = keras.models.load_model(f'model_weights/{model_name}-fold{fold_number}.keras')
		models.append(model)
	# Run simulations using trained models from each fold
	run_all(models, model_name, features, features_scaler, output_scaler, max_len)

get_results_for_model_size('ml_rnn') # Default to previously trained 16 unit model
get_results_for_model_size('ml_lstm') # Default to previously trained 16 unit model
get_results_for_model_size('ml_gru', 32)

ml_rnn
Average TRE RMSE: 0.40913350652486097
Average MTSK RMSE: 1.2484756718098815
ml_lstm
Average TRE RMSE: 0.4006921834540827
Average MTSK RMSE: 1.1933293403769494
ml_gru
Average TRE RMSE: 0.34778583558998105
Average MTSK RMSE: 0.9631366050031559


# TCN

In [10]:
# from tensorflow.keras.layers import Conv1D

# # MODEL NAME
# model_name = 'ml_tcn'

# model = Sequential()
# model.add(Input(shape=input_shape))
# model.add(Conv1D(16, kernel_size=4, activation='linear', padding='causal', kernel_regularizer=l2(0.01)))
# model.add(TimeDistributed(Dense(8, activation='linear', kernel_regularizer=l2(0.01))))
# model.add(Dropout(0.2))
# model.add(TimeDistributed(Dense(output_shape)))

# model.compile(loss='mse', optimizer='adam')

# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# model.fit(X_padded, y_padded, validation_split=0.2, epochs=500, batch_size=32, callbacks=[PrintCallback(), early_stopping], verbose=0)

# model.save('model_weights/{}.keras'.format(model_name))

# # Run simulations using trained model
# run_all(model, model_name, features, features_scaler, output_scaler, max_len)

Epoch: 500, loss: 0.03
ml_tcn
Average TRE RMSE: 0.4642991150064746
Average MTSK RMSE: 1.6289565766153424


# Seq2Seq

In [7]:
# from tensorflow.keras.layers import LSTM, Attention, Concatenate
# from tensorflow.keras.models import Model

# # MODEL NAME
# model_name = 'ml_seq2seq'

# encoder_inputs = Input(shape=input_shape)
# encoder_lstm = LSTM(16, return_sequences=True, return_state=True, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))
# encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# decoder_inputs = Input(shape=input_shape)
# decoder_lstm = LSTM(16, return_sequences=True, return_state=True, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))
# decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=[state_h, state_c])

# attention = Attention()([decoder_outputs, encoder_outputs])
# decoder_concat = Concatenate()([decoder_outputs, attention])
# decoder_dense = TimeDistributed(Dense(output_shape))
# decoder_outputs = decoder_dense(decoder_concat)

# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# model.compile(optimizer='adam', loss='mse')

# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# model.fit([X_padded, X_padded], y_padded, validation_split=0.2, epochs=500, batch_size=32, callbacks=[PrintCallback(), early_stopping], verbose=0)

# model.save('model_weights/{}.keras'.format(model_name))

# # Run simulations using trained model
# run_all(model, model_name, features, features_scaler, output_scaler, max_len, is_transformer=True)

Epoch: 185, loss: 0.03
ml_seq2seq
Average TRE RMSE: 0.4447912235248274
Average MTSK RMSE: 1.4460834447802842
